In [1]:
import warnings
warnings.filterwarnings("ignore")

import math
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, ExponentialLR, CosineAnnealingLR, StepLR, OneCycleLR

from dataloader import Dataset_AMEX
from metric import AmexMetric
from model_kaggle_ben import Transformer

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

import pandas as pd

In [2]:
# dataset = Dataset_AMEX('val')
# y_true = torch.tensor(dataset[0][1], dtype=torch.float)[None]

# model = Transformer(num_tokens=1,
#         feat_dim=188,
#         embed_dim = 64,
#         num_heads=4,
#         num_encoder_layers=2,
#         dropout_p=0.3)
# y_hats = model(torch.tensor(dataset[0][0])[None])
# loss_fn = nn.BCELoss(reduction="mean")
# loss_fn(y_hats.squeeze(1), y_true)
# #val_amex_metric = AmexMetric()
# #val_amex_metric.update(y_hats.reshape(-1), y_true)

# Data Module

In [3]:
class Dataset_pl(pl.LightningDataModule):
    def __init__(self, fold):
        super().__init__()
        self.fold = 1
        
    def prepare_data(self):
        pass

    def setup(self, stage= None):
        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            self.train_set = Dataset_AMEX('train', fold=self.fold)
            self.val_set = Dataset_AMEX('val', fold=self.fold)
        if stage == "validate":
            self.val_set = Dataset_AMEX('val', fold=self.fold)
        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.val_set = Dataset_AMEX('val', fold=self.fold)
        if stage == "predict" or stage is None:
            self.test_set = Dataset_AMEX('test')

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=512, shuffle=True, num_workers=4)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=2048, shuffle=False, num_workers=4)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=4096, shuffle=False, num_workers=4)

    def predict_dataloader(self):
        return DataLoader(self.test_set, batch_size=4096, shuffle=False, num_workers=4)

# Model

In [4]:
class Model_transformer(pl.LightningModule):
    def __init__(self, learning_rate=1e-3):#, batch_size):
        super().__init__()
        self.model = Transformer(num_tokens=1,
                        feat_dim=188,
                        embed_dim = 64,
                        num_heads=4,
                        num_encoder_layers=2,
                        dropout_p=0.3)
        self.learning_rate = learning_rate
        self.train_amex_metric = AmexMetric()
        self.val_amex_metric = AmexMetric()
        self.loss_fn = nn.BCEWithLogitsLoss(reduction="mean")
        
    def forward(self, x):
        # in lightning, forward defines the prediction/inference actions
        y_hat = self.model(x)
        return y_hat

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop. It is independent of forward
        x, y = batch
        x, y = x.float(), y.float()
        y_hat = self.model(x)
        # loss function
        loss = self.loss_fn(y_hat.squeeze(1), y)
        self.train_amex_metric.update(y_hat.squeeze(1), y)
        self.log_dict({'train_loss': loss, 'train_amex_metric': self.train_amex_metric}, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return {'loss': loss}
    
    def validation_step(self, batch, batch_idx):
        # training_step defines the train loop. It is independent of forward
        x, y = batch
        x, y = x.float(), y.float()
        y_hat = self.model(x)
        # loss function
        loss = self.loss_fn(y_hat.squeeze(1), y)
        self.val_amex_metric.update(y_hat.squeeze(1), y)
        self.log_dict({'val_loss': loss, 'val_amex_metric': self.val_amex_metric}, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return {'loss': loss}       

    def test_step(self, batch, batch_idx):
        # training_step defines the train loop. It is independent of forward
        x, y = batch
        x, y = x.float(), y.float()
        y_hat = self.model(x)
        # loss function
        #loss = self.loss_fn(y_hats.squeeze(1), y_true)

    def predict_step(self, batch, batch_idx):
        # training_step defines the train loop. It is independent of forward
        x, y = batch
        x, y = x.float(), y.float()
        with torch.no_grad():
            y_hat = self.model(x)#.squeeze(1)
        return y_hat
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
        #lr_scheduler = OneCycleLR(optimizer, max_lr=1e-3, epochs=25, steps_per_epoch=718)
        lr_scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=2, T_mult=3)
        return [optimizer], [lr_scheduler]

# Find LR

In [5]:
# dm = Dataset_pl(1)
# model = Model_transformer()#, argv['batch_size']) 
# trainer = pl.Trainer(gpus=2, strategy='dp')
# lr_finder = trainer.tuner.lr_find(model, dm)

# # Results can be found in
# lr_finder.results

# # Plot with
# fig = lr_finder.plot(suggest=True)
# fig.show()

# # Pick point based on plot, or get suggestion
# new_lr = lr_finder.suggestion()

# Training

In [6]:
# dm = Dataset_pl(1)
# model = Model_transformer()#, argv['batch_size']) 

# wandb_logger = WandbLogger(project="AMEX")
# callbacks=[ModelCheckpoint(dirpath='ckpt', 
#                            monitor="val_amex_metric", mode="max")]

# trainer = pl.Trainer(gpus=[1], max_epochs=30, 
#                     logger=wandb_logger, callbacks=callbacks,
#                     enable_progress_bar=False)

# trainer.fit(model, datamodule=dm)

# # get validation metrics
# val = trainer.validate(model, datamodule=dm, ckpt_path='best')
# val_amex_metric_epoch = val[0]['val_amex_metric_epoch']

# # get output
# output = trainer.predict(model, datamodule=dm, ckpt_path='best')
# output = torch.vstack(output)

In [7]:
prefix = 'coslr'

for i in range(1,11):
    dm = Dataset_pl(i)
    model = Model_transformer()#, argv['batch_size']) 

    #wandb_logger = WandbLogger(project="AMEX")
    callbacks=[ModelCheckpoint(dirpath='ckpt', 
                               monitor="val_amex_metric", mode="max")]

#     trainer = pl.Trainer(gpus=[1], max_epochs=25, 
#                         logger=wandb_logger, callbacks=callbacks,
#                         enable_progress_bar=True)
    trainer = pl.Trainer(gpus=1, max_epochs=25, 
                         callbacks=callbacks,
                         enable_progress_bar=True)
    trainer.fit(model, datamodule=dm)

    # get validation metrics
    val = trainer.validate(model, datamodule=dm, ckpt_path='best')
    val_amex_metric_epoch = val[0]['val_amex_metric_epoch']
    
    # get output
    output = trainer.predict(model, datamodule=dm, ckpt_path='best')
    output = torch.vstack(output)
    # save result
    df = pd.DataFrame(output)
    df2 = pd.read_csv('submission.csv')
    df2['prediction']=df.values
    df2.to_csv(f'results/{prefix}_prediction_fold{i}.csv', index=False)
    # save validation metrics to a txt file
    with open(f'results/{prefix}_fold_{i}_{val_amex_metric_epoch}.txt', 'x') as f:
        f.write(f"fold {i} {val_amex_metric_epoch}\n")
    del dm, trainer, model, output, df, df2
    import gc
    gc.collect()

    print(f"fold {i}", val_amex_metric_epoch)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


### Training data shapes (367131, 13, 188) (367131,)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type              | Params
--------------------------------------------------------
0 | model             | Transformer       | 509 K 
1 | train_amex_metric | AmexMetric        | 0     
2 | val_amex_metric   | AmexMetric        | 0     
3 | loss_fn           | BCEWithLogitsLoss | 0     
--------------------------------------------------------
509 K     Trainable params
0         Non-trainable params
509 K     Total params
2.037     Total estimated model params size (MB)


### Validation data shapes (91782, 13, 188) (91782,)
Epoch 24: 100%|██████████| 763/763 [00:09<00:00, 83.53it/s, loss=0.195, v_num=52, train_loss_step=0.295, val_loss_step=0.244, val_loss_epoch=0.243, val_amex_metric_epoch=0.767, train_loss_epoch=0.186, train_amex_metric_epoch=0.837]


Restoring states from the checkpoint path at /home/cairs/code/amex/transformer/ckpt/epoch=6-step=5026-v5.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /home/cairs/code/amex/transformer/ckpt/epoch=6-step=5026-v5.ckpt


...

### Validation data shapes (91782, 13, 188) (91782,)
Epoch 24: 100%|██████████| 763/763 [00:09<00:00, 81.33it/s, loss=0.188, v_num=61, train_loss_step=0.118, val_loss_step=0.238, val_loss_epoch=0.235, val_amex_metric_epoch=0.772, train_loss_epoch=0.194, train_amex_metric_epoch=0.827]


Restoring states from the checkpoint path at /home/cairs/code/amex/transformer/ckpt/epoch=7-step=5744-v23.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /home/cairs/code/amex/transformer/ckpt/epoch=7-step=5744-v23.ckpt


### Validation data shapes (91782, 13, 188) (91782,)
Validation DataLoader 0: 100%|██████████| 45/45 [00:00<00:00, 107.71it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
  val_amex_metric_epoch     0.7818259291465592
     val_loss_epoch         0.22407042980194092
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


100%|██████████| 20/20 [00:02<00:00,  7.85it/s]
Restoring states from the checkpoint path at /home/cairs/code/amex/transformer/ckpt/epoch=7-step=5744-v23.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /home/cairs/code/amex/transformer/ckpt/epoch=7-step=5744-v23.ckpt


### Test data shapes (924621, 13, 188) (210,)
Predicting DataLoader 0: 100%|██████████| 226/226 [00:05<00:00, -84.69it/s]  
fold 10 0.7818259291465592
